<a href="https://colab.research.google.com/github/marcin-sielski/google_colab_notebooks/blob/master/PyTorch_VNC_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch VNC Colab

This Colab Notebook is meant to install Intel® OpenVINO™ and pre-configure Model Optimizer with TensorFlow framework.

1.   Make sure at ``Edit/Notebook settings`` that ``Hardware accelerator`` is set to ``GPU``.
2.   Sign-up for free account at https://ngrok.com/ and pickup authtoken (3. Connect your account),
3.   Modify ``NGROK_AUTH`` at first execution cell accordingly,
4.   ``Run All`` at Colab Notebook,
5.   Colab Notebook will ask for your authorization to Google Drive (enables to access to your models storage),
6.   Colab Notebook will ask what password shall be set to your VNC session,
7.   Once Colab Notebook execution is completed scroll down to get ngrok link,
8.   Open VNC Viewer and use ngrok link to connect to your Colab Machine,
9.   Use previously entered password for VNC session.
10.   Select ``Use default config`` button, 
11.   Open terminal,
12.   Enjoy!!! 



In [0]:
NGROK_AUTH = ""

In [0]:
!apt update
!apt install pciutils
!/usr/bin/lspci
!cat /proc/cpuinfo
!cat /proc/meminfo
!nvidia-smi

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [81.6 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 https://developer.

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!apt update
# install xface
!apt install xfce4 xfce4-goodies
# install vnc server
!apt install vnc4server
# install ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
70 packages can be upgraded. Run 'a

In [0]:
!vncpasswd

Password:
Verify:


In [0]:
!vnc4server
!vnc4server -kill :1

xauth:  file /root/.Xauthority does not exist

New '28228350e059:1 ()' desktop is 28228350e059:1

Creating default startup script /root/.vnc/xstartup
Starting applications specified in /root/.vnc/xstartup
Log file is /root/.vnc/28228350e059:1.log

Killing Xvnc4 process ID 8261


In [0]:
!sed -i 's/# unset SESSION_MANAGER/unset SESSION_MANAGER/g' /root/.vnc/xstartup
!sed -i 's/# exec \/etc\/X11\/xinit\/xinitrc/exec \/etc\/X11\/xinit\/xinitrc/g' /root/.vnc/xstartup
!cat /root/.vnc/xstartup

#!/bin/sh

# Uncomment the following two lines for normal desktop:
unset SESSION_MANAGER
exec /etc/X11/xinit/xinitrc

[ -x /etc/vnc/xstartup ] && exec /etc/vnc/xstartup
[ -r $HOME/.Xresources ] && xrdb $HOME/.Xresources
xsetroot -solid grey
vncconfig -iconic &
x-terminal-emulator -geometry 80x24+10+10 -ls -title "$VNCDESKTOP Desktop" &
x-window-manager &


In [0]:
VNC_PORT = 5901

from time import sleep
from IPython.display import clear_output
import requests
import json

# keep everything running
i=0
while True:
  try:
    # start vnc server
    if len(get_ipython().getoutput('ps -ef | grep vnc')) <= 2:
      get_ipython().system_raw('vnc4server -httpport $VNC_PORT &')
      sleep(5)
  
    ngrok_link = json.loads(requests.get('http://localhost:4040/api/tunnels').text)['tunnels'][0]['public_url']
    print(ngrok_link)
    sleep(30)
    requests.get(ngrok_link)
  except (KeyboardInterrupt, SystemExit):
      raise
  except:
    # create ngrok port
    get_ipython().system_raw('./ngrok authtoken $NGROK_AUTH && ./ngrok tcp $VNC_PORT &')
  i+=1
  if i >=5:
    clear_output(wait=True)
    i=0

tcp://0.tcp.ngrok.io:19847
tcp://0.tcp.ngrok.io:19847
tcp://0.tcp.ngrok.io:19847
tcp://0.tcp.ngrok.io:19847
tcp://0.tcp.ngrok.io:19847
